In [8]:
import os
from pathlib import Path

import librosa
import matplotlib.pyplot as plt
# import mne
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from IPython.display import Audio

# from datasets import DoubleDataset
from nn_modules import BrainModule
# from trainer import Trainer

In [2]:
dirprocess = '../Spatial-Attention/data/process_v2/'

hidden_test = np.load(os.path.join(dirprocess, 'audio/extract_features_test4.npy'))

df_test = pd.read_csv(os.path.join(dirprocess, f'dataframe/df_test27.csv'))

meg = dict(np.load(os.path.join(
    dirprocess, 'meg/meg27_sr100_default_v1.npz'
)))

In [11]:
df_test.head(15)

,subject_id,session_id,story_id,sound_id,onset_meg,sound_length,sound_fname,segment_start,words,meg100_start,meg100_stop,meg120_start,meg120_stop,meg200_start,meg200_stop,wav_start,wav_stop,wav_index
0,1,0,3,5,739.549,81.943311,the_black_willow_5.wav,0,['hard' 'current' 'cheeks' 'understand'],73955,74255,88746,89106,147910,148510,0,48000,0
1,1,0,3,5,739.549,81.943311,the_black_willow_5.wav,1,['understand' 'You'],74055,74355,88866,89226,148110,148710,16000,64000,1
2,1,0,3,5,739.549,81.943311,the_black_willow_5.wav,2,['You' 'mean' 'finished'],74155,74455,88986,89346,148310,148910,32000,80000,2
3,1,0,3,5,739.549,81.943311,the_black_willow_5.wav,3,['You' 'mean' 'finished' 'another' 'story'],74255,74555,89106,89466,148510,149110,48000,96000,3
4,1,0,3,5,739.549,81.943311,the_black_willow_5.wav,4,['finished' 'another' 'story'],74355,74655,89226,89586,148710,149310,64000,112000,4
5,1,0,3,5,739.549,81.943311,the_black_willow_5.wav,5,['story' 'Yes'],74455,74755,89346,89706,148910,149510,80000,128000,5
6,1,0,3,5,739.549,81.943311,the_black_willow_5.wav,6,['Yes' 'it' 'is' 'finished'],74555,74855,89466,89826,149110,149710,96000,144000,6
7,1,0,3,5,739.549,81.943311,the_black_willow_5.wav,7,['Yes' 'it' 'is' 'finished' 'Ended'],74655,74955,89586,89946,149310,149910,112000,160000,7
8,1,0,3,5,739.549,81.943311,the_black_willow_5.wav,8,['it' 'is' 'finished' 'Ended'],74755,75055,89706,90066,149510,150110,128000,176000,8
9,1,0,3,5,739.549,81.943311,the_black_willow_5.wav,9,['Ended' 'tears' 'birch'],74855,75155,89826,90186,149710,150310,144000,192000,9


In [20]:
row = df_test.iloc[500]
print(row)

sound_fname = row['sound_fname']
wav_start = row['wav_start']
wav_stop = row['wav_stop']
wav_index = row['wav_index']

audio, sr = librosa.load(f'../Spatial-Attention/data/bids_anonym/stimuli/audio/{sound_fname}', sr=None)
sample = audio[wav_start:wav_stop]
Audio(sample, rate=sr)

subject_id                            1
session_id                            0
story_id                              3
sound_id                              8
onset_meg                       1170.58
sound_length                  142.41424
sound_fname      the_black_willow_8.wav
segment_start                       127
words               ['Nathan' 'aghast']
meg100_start                     129758
meg100_stop                      130058
meg120_start                     155710
meg120_stop                      156070
meg200_start                     259516
meg200_stop                      260116
wav_start                       2032000
wav_stop                        2080000
wav_index                           500
Name: 500, dtype: object


In [16]:
hidden_test.shape, df_test.shape

((999, 149, 512), (48951, 18))

In [18]:
len(set(df_test['wav_index']))

999

In [19]:
df_test.iloc[998:1000]

,subject_id,session_id,story_id,sound_id,onset_meg,sound_length,sound_fname,segment_start,words,meg100_start,meg100_stop,meg120_start,meg120_stop,meg200_start,meg200_stop,wav_start,wav_stop,wav_index
998,1,0,3,11,1735.437,119.185533,the_black_willow_11.wav,115,['the' 'water' 'of' 'the' 'stream' 'flowing' '...,185044,185344,222052,222412,370087,370687,1840000,1888000,998
999,1,1,3,5,698.994,81.943311,the_black_willow_5.wav,0,['hard' 'current' 'cheeks' 'understand'],69899,70199,83879,84239,139799,140399,0,48000,0


In [ ]:
model = BrainModule(
    n_channels_input=208,
    n_channels_attention=270,
    n_channels_unmix=10,
    use_spatial_attention='3D',
    n_spatial_harmonics=24,
    dirprocess=dirprocess,
    spatial_dropout_number=0,
    spatial_dropout_radius=0.1,
    use_unmixing_layer=True,
    use_subject_layer=True,
    n_subjects=27,
    regularize_subject_layer=False,
    bias_subject_layer=False,
    n_channels_block=320,
    n_features=512,
    head_pool='conv',
    head_stride=2,
    meg_sr=100,
    use_temporal_filter=True,
    use_temporal_activation=False,
    use_unmixing_bias=False,
    use_temporal_bias=False
)
checkpoint = torch.load('checkpoint.pt', weights_only=True, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [ ]:
def get_meg_data():
    meg_offset = int(meg_offset * meg_sr)

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        # TODO: .loc works on labels, .iloc on positions of rows. Alex says he always resets index (label) and they
        #  coincide. However, might be worth updating to avoid potential errors in the future
        row_df = self.df.loc[index]

        subject_id, session_id, story_id = row_df['subject_id'], row_df['session_id'], row_df['story_id']
        sbj = torch.tensor(subject_id, dtype=torch.long)
        subject_id = str(subject_id)
        subject_id = '0' + subject_id if len(subject_id) == 1 else subject_id
        subset = f'subject{subject_id}_session{session_id}_story{story_id}'

        meg_start, meg_stop = row_df[f'meg{self.meg_sr}_start'], row_df[f'meg{self.meg_sr}_stop']
        meg_start, meg_stop = meg_start + self.meg_offset, meg_stop + self.meg_offset
        wav_index = row_df['wav_index']

        meg = torch.tensor(self.meg[subset][:, meg_start:meg_stop], dtype=torch.float32)
        hid = torch.tensor(self.hidden[wav_index], dtype=torch.float32)
        widx = torch.tensor(wav_index, dtype=torch.long)

        return meg, sbj, hid, widx
